In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
ROM = pd.read_csv("C:/Users/user/data.csv") 
X = np.array(ROM.drop('ROM_worsing',1))
y = np.array(ROM['ROM_worsing'])

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from math import sqrt
stkf = StratifiedKFold(n_splits=5)
mmscaler = preprocessing.MinMaxScaler() 
select7 = RFE(RandomForestClassifier(n_estimators=300), n_features_to_select=7, verbose=1)
select6 = RFE(RandomForestClassifier(n_estimators=300), n_features_to_select=6, verbose=1)
select5 = RFE(RandomForestClassifier(n_estimators=300), n_features_to_select=5, verbose=1)
select4 = RFE(RandomForestClassifier(n_estimators=300), n_features_to_select=4, verbose=1)
select3 = RFE(RandomForestClassifier(n_estimators=300), n_features_to_select=3, verbose=1)

# Logistic regression
from sklearn.linear_model import LogisticRegression
log_params = {'C':[0.01,0.1,1,10,100]}
log = LogisticRegression(multi_class = 'auto', max_iter=500)

# Support vector machine
from sklearn.svm import SVC
svc_params = {'C':[100,1000], 'gamma':[1,0.1,0.01,0.001]}
svc = SVC(kernel='rbf', max_iter=500)

# MLP classifier
from sklearn.neural_network import MLPClassifier
mlp_params = {'hidden_layer_sizes':[100,100,100], 'learning_rate_init':[0.1,0.01,0.001]}
mlp = MLPClassifier(activation='relu' ,solver='adam', max_iter=500)

# Decition tree
from sklearn.tree import DecisionTreeClassifier
tree_params = {'max_depth':[2,3,4,5,10,15], 'min_samples_leaf':[5,10,15], 'min_samples_split':[2,3,4,5]}
tree = DecisionTreeClassifier()

# Random forest
from sklearn.ensemble import RandomForestClassifier
forest_params = {'n_estimators':[500,800], 'criterion': ['entropy'], 'max_depth':[2,3,4,5,10,15],'bootstrap': ['False','True']}
forest = RandomForestClassifier()

lists=[]
for train_index, test_index in stkf.split(X, y):
    # Recursive Feature Elimination with random forest
    select5.fit(X[train_index], y[train_index])
    X_train_sel = select5.transform(X[train_index])
    X_test_sel = select5.transform(X[test_index])
    
    # Logistic regression
    gslog = GridSearchCV(log, log_params, cv=stkf) 
    gslog.fit(X_train_sel, y[train_index])
    best = gslog.best_estimator_
    best_y = best.predict(X_test_sel)
    log_accuracy = accuracy_score(y[test_index], best_y)
    lists.append(log_accuracy)
    log_auc = roc_auc_score(y[test_index], best_y)
    lists.append(log_auc)
    N1 = sum(y[test_index] == 1)
    N2 = sum(y[test_index] != 1)
    Q1 = log_auc / (2 - log_auc)
    Q2 = 2*log_auc**2 / (1 + log_auc)
    SE_AUC = sqrt((log_auc*(1 - log_auc) + (N1 - 1)*(Q1 - log_auc**2) + (N2 - 1)*(Q2 - log_auc**2)) / (N1*N2))
    lower = log_auc - 1.96*SE_AUC
    lists.append(lower)
    upper = log_auc + 1.96*SE_AUC
    lists.append(upper)
    
    # SVM 
    gssvc = GridSearchCV(svc, svc_params, cv=stkf)
    gssvc.fit(X_train_sel, y[train_index])
    best = gssvc.best_estimator_
    best_y = best.predict(X_test_sel)
    svc_accuracy = accuracy_score(y[test_index], best_y)
    lists.append(svc_accuracy)
    svc_auc = roc_auc_score(y[test_index], best_y)
    lists.append(svc_auc)
    N1 = sum(y[test_index] == 1)
    N2 = sum(y[test_index] != 1)
    Q1 = svc_auc / (2 - svc_auc)
    Q2 = 2*svc_auc**2 / (1 + svc_auc)
    SE_AUC = sqrt((svc_auc*(1 - svc_auc) + (N1 - 1)*(Q1 - svc_auc**2) + (N2 - 1)*(Q2 - svc_auc**2)) / (N1*N2))
    lower = svc_auc - 1.96*SE_AUC
    lists.append(lower)
    upper = svc_auc + 1.96*SE_AUC
    lists.append(upper)
    
    # MLP classifier
    gsmlp = GridSearchCV(mlp, mlp_params, cv=stkf) 
    gsmlp.fit(X_train_sel, y[train_index])
    best = gsmlp.best_estimator_
    best_y = best.predict(X_test_sel)
    mlp_accuracy = accuracy_score(y[test_index], best_y)
    lists.append(mlp_accuracy)
    mlp_auc = roc_auc_score(y[test_index], best_y)
    lists.append(mlp_auc)
    N1 = sum(y[test_index] == 1)
    N2 = sum(y[test_index] != 1)
    Q1 = mlp_auc / (2 - mlp_auc)
    Q2 = 2*mlp_auc**2 / (1 + mlp_auc)
    SE_AUC = sqrt((mlp_auc*(1 - mlp_auc) + (N1 - 1)*(Q1 - mlp_auc**2) + (N2 - 1)*(Q2 - mlp_auc**2)) / (N1*N2))
    lower = mlp_auc - 1.96*SE_AUC
    lists.append(lower)
    upper = mlp_auc + 1.96*SE_AUC
    lists.append(upper)
    
    # Decition tree
    gstree = GridSearchCV(tree, tree_params, cv=stkf) 
    gstree.fit(X_train_sel, y[train_index])
    best = gstree.best_estimator_
    best_y = best.predict(X_test_sel)
    tree_accuracy = accuracy_score(y[test_index], best_y)
    lists.append(tree_accuracy)
    tree_auc = roc_auc_score(y[test_index], best_y)
    lists.append(tree_auc)
    N1 = sum(y[test_index] == 1)
    N2 = sum(y[test_index] != 1)
    Q1 = tree_auc / (2 - tree_auc)
    Q2 = 2*tree_auc**2 / (1 + tree_auc)
    SE_AUC = sqrt((tree_auc*(1 - tree_auc) + (N1 - 1)*(Q1 - tree_auc**2) + (N2 - 1)*(Q2 - tree_auc**2)) / (N1*N2))
    lower = tree_auc - 1.96*SE_AUC
    lists.append(lower)
    upper = tree_auc + 1.96*SE_AUC
    lists.append(upper)
    
    # Five variables
    gsforest = GridSearchCV(forest, forest_params, cv=stkf) 
    gsforest.fit(X_train_sel, y[train_index])
    best = gsforest.best_estimator_
    best_y = best.predict(X_test_sel)
    forest_accuracy = accuracy_score(y[test_index], best_y)
    lists.append(forest_accuracy)
    forest_auc = roc_auc_score(y[test_index], best_y)
    lists.append(forest_auc)
    N1 = sum(y[test_index] == 1)
    N2 = sum(y[test_index] != 1)
    Q1 = forest_auc / (2 - forest_auc)
    Q2 = 2*forest_auc**2 / (1 + forest_auc)
    SE_AUC = sqrt((forest_auc*(1 - forest_auc) + (N1 - 1)*(Q1 - forest_auc**2) + (N2 - 1)*(Q2 - forest_auc**2)) / (N1*N2))
    lower = forest_auc - 1.96*SE_AUC
    lists.append(lower)
    upper = forest_auc + 1.96*SE_AUC
    lists.append(upper)
    
    # Six variables
    select6.fit(X[train_index], y[train_index])
    X_train_sel = select6.transform(X[train_index])
    X_test_sel = select6.transform(X[test_index])
    gsforest = GridSearchCV(forest, forest_params, cv=stkf) 
    gsforest.fit(X_train_sel, y[train_index])
    best = gsforest.best_estimator_
    best_y = best.predict(X_test_sel)
    forest_accuracy = accuracy_score(y[test_index], best_y)
    lists.append(forest_accuracy)
    forest_auc = roc_auc_score(y[test_index], best_y)
    lists.append(forest_auc)
    N1 = sum(y[test_index] == 1)
    N2 = sum(y[test_index] != 1)
    Q1 = forest_auc / (2 - forest_auc)
    Q2 = 2*forest_auc**2 / (1 + forest_auc)
    SE_AUC = sqrt((forest_auc*(1 - forest_auc) + (N1 - 1)*(Q1 - forest_auc**2) + (N2 - 1)*(Q2 - forest_auc**2)) / (N1*N2))
    lower = forest_auc - 1.96*SE_AUC
    lists.append(lower)
    upper = forest_auc + 1.96*SE_AUC
    lists.append(upper)
    
    # Seven variables
    select7.fit(X[train_index], y[train_index])
    X_train_sel = select7.transform(X[train_index])
    X_test_sel = select7.transform(X[test_index])
    gsforest = GridSearchCV(forest, forest_params, cv=stkf) 
    gsforest.fit(X_train_sel, y[train_index])
    best = gsforest.best_estimator_
    best_y = best.predict(X_test_sel)
    forest_accuracy = accuracy_score(y[test_index], best_y)
    lists.append(forest_accuracy)
    forest_auc = roc_auc_score(y[test_index], best_y)
    lists.append(forest_auc)
    N1 = sum(y[test_index] == 1)
    N2 = sum(y[test_index] != 1)
    Q1 = forest_auc / (2 - forest_auc)
    Q2 = 2*forest_auc**2 / (1 + forest_auc)
    SE_AUC = sqrt((forest_auc*(1 - forest_auc) + (N1 - 1)*(Q1 - forest_auc**2) + (N2 - 1)*(Q2 - forest_auc**2)) / (N1*N2))
    lower = forest_auc - 1.96*SE_AUC
    lists.append(lower)
    upper = forest_auc + 1.96*SE_AUC
    lists.append(upper)
    
    # Four variables
    select4.fit(X[train_index], y[train_index])
    X_train_sel = select4.transform(X[train_index])
    X_test_sel = select4.transform(X[test_index])
    gsforest = GridSearchCV(forest, forest_params, cv=stkf) 
    gsforest.fit(X_train_sel, y[train_index])
    best = gsforest.best_estimator_
    best_y = best.predict(X_test_sel)
    forest_accuracy = accuracy_score(y[test_index], best_y)
    lists.append(forest_accuracy)
    forest_auc = roc_auc_score(y[test_index], best_y)
    lists.append(forest_auc)
    N1 = sum(y[test_index] == 1)
    N2 = sum(y[test_index] != 1)
    Q1 = forest_auc / (2 - forest_auc)
    Q2 = 2*forest_auc**2 / (1 + forest_auc)
    SE_AUC = sqrt((forest_auc*(1 - forest_auc) + (N1 - 1)*(Q1 - forest_auc**2) + (N2 - 1)*(Q2 - forest_auc**2)) / (N1*N2))
    lower = forest_auc - 1.96*SE_AUC
    lists.append(lower)
    upper = forest_auc + 1.96*SE_AUC
    lists.append(upper)
    
    # Three variables
    select3.fit(X[train_index], y[train_index])
    X_train_sel = select3.transform(X[train_index])
    X_test_sel = select3.transform(X[test_index])
    gsforest = GridSearchCV(forest, forest_params, cv=stkf) 
    gsforest.fit(X_train_sel, y[train_index])
    best = gsforest.best_estimator_
    best_y = best.predict(X_test_sel)
    forest_accuracy = accuracy_score(y[test_index], best_y)
    lists.append(forest_accuracy)
    forest_auc = roc_auc_score(y[test_index], best_y)
    lists.append(forest_auc)
    N1 = sum(y[test_index] == 1)
    N2 = sum(y[test_index] != 1)
    Q1 = forest_auc / (2 - forest_auc)
    Q2 = 2*forest_auc**2 / (1 + forest_auc)
    SE_AUC = sqrt((forest_auc*(1 - forest_auc) + (N1 - 1)*(Q1 - forest_auc**2) + (N2 - 1)*(Q2 - forest_auc**2)) / (N1*N2))
    lower = forest_auc - 1.96*SE_AUC
    lists.append(lower)
    upper = forest_auc + 1.96*SE_AUC
    lists.append(upper)
lists

# Feature importance
data=pd.DataFrame(lists)
data.to_csv("C:/Users/user/data.csv", encoding = 'shift-jis')
lists=[]
for train_index, test_index in stkf.split(X, y):
    forest.fit(X[train_index], y[train_index])
    fi = forest.feature_importances_
    lists.append(fi)
data=pd.DataFrame(lists)
mfi = mmscaler.fit_transform(data.T)
mfi = pd.DataFrame(mfi, columns=['a','b','c','d','e'])
meanfi = mfi.mean(axis='columns')
meanfi = pd.DataFrame(meanfi, columns=['mean'])
data = pd.concat([mfi, meanfi], axis=1)
data.index = 'age', 'female/male', 'right/left', 'OA', 'RA', 'history', 'bmi', 'hba1c', 'mua', 'logic/others', 'persona/others', 'optetrak/others','fine/others', 'cr/other', 'mpp/lpp', 'fta_pre', 'fta_po','flex_act_pre', 'flex_act_4w', 'flex_pas_pre', 'flex_pas_4w','ext_act_pre', 'ext_act_4w', 'ext_pas_pre', 'ext_pas_4w','gait_nor_pre', 'gait_nor_4w', 'gait_max_pre', 'gait_max_4w', 'joa_pre','joa_4w', 'α', 'β', 'θ', 'γ', 'JL_change'
data.to_csv("C:/Users/user/feature_importance.csv", encoding = 'shift-jis')